In [1]:
import collections
import re
from d2l import torch as d2l

In [2]:
#首先需要读取数据集
d2l.DATA_HUB['time_machine']=(d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')
#定义读取时间机器数据集的函数,并将其中所有的非字母替换成空格，并且全部改成小写
def read_time_machine():
    with open(d2l.download('time_machine'),'r') as f:
        lines=f.readlines()
    
    return [re.sub('[^A-Za-z]+',' ',line).strip().lower() for line in lines ]

lines=read_time_machine()
print(f'# 文本总行数：{len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数：3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [3]:
#接下来进行词元化
def tokenize(lines,token='word'):
    #如果是单词，就将行按照空格分开
    if token=='word':
        return [line.split() for line in lines]
    #如果是字母，就将行转成列表
    elif token=='char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：'+token)

tokens= tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [4]:
#计算词元出现的频率
def count_corpus(tokens):
    #这里的tokens是1D列表或者2D列表
    if len(tokens)==0 or isinstance(tokens[0],list):
        #将次元列表战平成一个列表,这里的line是2维的列表中的每行的意思
        tokens=[token for line in tokens for token in line]
    #返回词元和出现频次的数据结构
    return collections.Counter(tokens)

#定义词表类
class Vocab:
    def __init__(self,tokens=None,min_freq=0,reserved_tokens=None):
        #如果词元和保留词元没有给出的话，就给他们初始化为空列表
        if tokens is None:
            tokens=[]
        if reserved_tokens is None:
            reserved_tokens=[]
        
        #按照出现的频率进行排序
        counter=count_corpus(tokens)
        #按降序排列，频次高的在前面
        self._token_freqs=sorted(counter.items(),key=lambda x:x[1],reverse=True )

        #未知的词元的索引为0
        self.idx_to_token=['<unk>']+reserved_tokens
        #保留词元对应的索引
        self.token_to_idx={token:idx for idx,token in enumerate(self.idx_to_token)}

        for token,freq in self._token_freqs:
            #如果该词元出现的频次比较低，就直接删除
            if freq<min_freq:
                break
            #如果当前的词元没有出现在已经记录的词元列表中，则要放进去
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token]=len(self.idx_to_token)-1

    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self,tokens):
        #获取给出词元的索引，如果获取不到的话，就说明是unk词元，返回self.unk就可以了
        if not isinstance(tokens,(list,tuple)):
            return self.token_to_idx.get(tokens,self.unk)
        #否则递归调用自身
        return [self.__getitem__(token) for token in tokens]
    
    #获取索引对应的词元
    def to_tokens(self,indices):
        if not isinstance(indices,(list,tuple)):
            return self.idx_to_token[indices]

        return[self.idx_to_token[index] for index in indices] 

    def unk():
        return 0

    def token_freqs(self):
        return self._token_freqs
        

In [5]:
#首先使用时光机器数据集作为语料库来构建词表，然后打印前几个高频词元，及其索引
vocab=Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [6]:
#接下来可以把每一条文本行转换成一个数字索引列表
for i in [0,10]:
    print('文本：',tokens[i])
    print('索引：',vocab[tokens[i]])

文本： ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引： [1, 19, 50, 40, 2183, 2184, 400]
文本： ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引： [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [10]:
#整合上述所有的功能,返回时光机器数据集的词元索引表和词表
def load_corpus_time_machine(max_tokens=-1):
    lines=read_time_machine()
    tokens=tokenize(lines,'char')
    vocab=Vocab(tokens)
    #因为这个数据集中每个文本行不一定是一个句子或者段落，所以将所有文本行展平到一个列表中
    corpus=[vocab[token] for line in tokens for token in line]
    if max_tokens>0:
        corpus=corpus[:max_tokens]
    return corpus,vocab

corpus,vocab=load_corpus_time_machine()
len(corpus),len(vocab)

(170580, 28)